## tf.function 和 自动图计算

1. 可以将普通的Python代码变成Tensorflow中的图实现的函数
2. Tensorflow 对图实现的函数有优化
3. 本节主要作用是保存模型和读取模型

In [1]:
import tensorflow.keras as keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as  plt
% matplotlib inline
import numpy as np
import sklearn
import os
import sys
import time
import tensorflow as tf

In [2]:
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.3.0
sys.version_info(major=3, minor=7, micro=11, releaselevel='final', serial=0)
matplotlib 3.4.2
numpy 1.18.5
pandas 1.3.3
sklearn 1.0
tensorflow 2.3.0
tensorflow.keras 2.4.0


### 1. tf.function的使用

- tf.greater_equal(a, b) 判断ab是否相等
- tf.where(is_position, z, alpha * tf.nn.relu(z)) 条件
- %timeit scaled_elu(tf.random.normal((1000, 1000))) 计算代码运行的时间

In [10]:
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_position = tf.greater_equal(z, 0.0)  # 判断两个值是否相等
    return scale * tf.where(is_position, z, alpha * tf.nn.elu(z))

In [11]:
print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3., -2.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)


In [14]:
# 将Python函数的代码, 转换为 Tensorflow的图实现的函数

tf_scaled_elu = tf.function(scaled_elu)

print(tf_scaled_elu(tf.constant(-3.)))
print(tf_scaled_elu(tf.constant([-3., -2.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)


In [17]:
# 通过python_function 找到对应的python方法
print("tf_scaled_elu == scaled_elu  : ", tf_scaled_elu.python_function is scaled_elu)

tf_scaled_elu == scaled_elu  :  True


In [18]:
% timeit scaled_elu(tf.random.normal((1000, 1000)))
% timeit tf_scaled_elu(tf.random.normal((1000, 1000)))

26.4 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
24.1 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. @tf.function
- 在函数前面加上 tf.function 就可以把函数变成tf中图实现的函数

In [19]:
@tf.function
def converge_to_2(n_iters):
    """
        1 + 1 / 2 + 1 / 2^2 + ...
    :param n_iters:
    :return:
    """
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

In [20]:
print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


### 3. 将代码转换为Tensorflow代码

In [25]:
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [28]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
    with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        is_position = ag__.converted_call(ag__.ld(tf).greater_equal, (ag__.ld(z), 0.0), None, fscope)
        try:
            do_return = True
            retval_ = (ag__.ld(scale) * ag__.converted_call(ag__.ld(tf).where, (ag__.ld(is_position), ag__.ld(z), (ag__.ld(alpha) * ag__.converted_call(ag__.ld(tf).nn.elu, (ag__.ld(z),), None, fscope))), None, fscope))
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

### 4. 使用时的注意事项

- tf.Variable(0. ) 如果在 @tf.function 会报错

In [33]:
var = tf.Variable(0.)


@tf.function
def add_21():
    return var.assign_add(21)


print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


### 5. 函数签名

In [53]:
# tf.TensorSpec(shape, dtype, minimum, maximum, name=None)
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name="z")])
def cube(z):
    return tf.pow(z, 2)

In [57]:
try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)

print()
print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='z'))

tf.Tensor([1 4 9], shape=(3,), dtype=int32)


### 6. 图结构

- get_concrete_function -> 为函数添加 输入签名 => 让函数成为一个可以保存的 Model

In [59]:
# get_concrete_function -> 为函数添加 输入签名 => 让函数成为一个可以保存的 Model
# 使用时需要传入函数签名
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32, name="x"))
print(cube_func_int32)

ConcreteFunction cube(z)
  Args:
    z: int32 Tensor, shape=(None,)
  Returns:
    int32 Tensor, shape=(None,)


In [60]:
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([None], tf.int32, name="x")))

True


In [62]:
# 打印图定义
cube_func_int32.graph

In [63]:
# 获取函数所有操作
cube_func_int32.graph.get_operations()

[<tf.Operation 'z' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [66]:
# Placeholder 用来放输入数据
cube_func_int32.graph.get_operation_by_name("z")

<tf.Operation 'z' type=Placeholder>

In [68]:
# 获取tensor
cube_func_int32.graph.get_tensor_by_name("z:0")

<tf.Tensor 'z:0' shape=(None,) dtype=int32>

In [69]:
# 打印 graph_def
cube_func_int32.graph.as_graph_def()

node {
  name: "z"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "z"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 2
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "z"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 440
}